In [ ]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

# setting env variable

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads .env in current working dir

key = os.getenv("OPENAI_API_KEY")
if not key:
    raise RuntimeError("OPENAI_API_KEY not found. Add it to a .env file or set the environment variable.")

os.environ["OPENAI_API_KEY"] = key

# Data Ingestion

In [9]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("D:\\PW\\LLMOPS\\data\\AgenticAI.txt", encoding="utf8")
documents = loader.load()

In [10]:
documents[0].page_content[:500]  # Print the first 500 characters of the first documen

'Understanding Agentic AI\n\nAgentic AI refers to a new paradigm in artificial intelligence where systems are designed not just to respond to queries or perform specific \ntasks, but to operate autonomously towards achieving predefined goals. This involves capabilities such as planning, reasoning, executing actions, \nand continuously adapting to dynamic environments without constant human intervention.\n\nKey Characteristics of Agentic AI:\n• Goal-oriented: They possess a clear objective or goal that g'

In [16]:
! uv pip install langchain-text-splitters

Using Python 3.11.9 environment at: D:\PW\LLMOPS\.venv
Audited 1 package in 42ms


In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
text_chunks=text_splitter.split_documents(documents)

In [24]:
text_chunks[1]  # Print the first chunk

Document(metadata={'source': 'D:\\PW\\LLMOPS\\data\\AgenticAI.txt'}, page_content='Agentic AI refers to a new paradigm in artificial intelligence where systems are designed not just to respond to queries or perform specific')

In [25]:
! uv pip install faiss-cpu

Using Python 3.11.9 environment at: D:\PW\LLMOPS\.venv
Resolved 3 packages in 703ms
 Downloaded faiss-cpu
Prepared 1 package in 3.08s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 193ms
 + faiss-cpu==1.13.0


In [26]:
! uv pip install langchain-openai

Using Python 3.11.9 environment at: D:\PW\LLMOPS\.venv
Resolved 33 packages in 1.00s
Prepared 1 package in 188ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 110ms
 + langchain-openai==1.1.0


In [27]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [28]:
embeddings = OpenAIEmbeddings()

In [29]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [30]:
vectorstore

In [32]:
retriever = vectorstore.as_retriever()

In [33]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

Document 1:
Key Characteristics of Agentic AI:
• Goal-oriented: They possess a clear objective or goal that guides their actions.
--------------------------------------------------
Document 2:
Understanding Agentic AI
--------------------------------------------------
Document 3:
How Agentic AI Works:
The typical workflow of an agentic AI system involves a continuous loop of perception, planning, action, and learning.
--------------------------------------------------
Document 4:
Applications of Agentic AI:
• Robotics: Autonomous robots capable of complex tasks in manufacturing, exploration, or logistics.
--------------------------------------------------


In [35]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt=ChatPromptTemplate.from_template(template)

In [36]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

output_parser=StrOutputParser()

llm_model=ChatOpenAI(model_name="gpt-4o-mini")

In [41]:
from langchain_core.runnables import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [42]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI refers to a type of artificial intelligence that is goal-oriented and designed to achieve specific objectives. It operates through a continuous loop of perception, planning, action, and learning, allowing it to adjust its behavior based on feedback from its environment. This adaptability is crucial for effectively reaching its goals. Common applications of Agentic AI include robotics, where autonomous robots perform complex tasks such as manufacturing, exploration, and logistics. The defining characteristic of Agentic AI is its clear objective that guides its actions, distinguishing it from other AI types that may lack such a focused purpose. Overall, Agentic AI systems are designed to act independently and intelligently in dynamic environments.'

'Agentic AI refers to a type of artificial intelligence that is goal-oriented and designed to achieve specific objectives. It operates through a continuous loop of perception, planning, action, and learning, allowing it to adjust its behavior based on feedback from its environment. This adaptability is crucial for effectively reaching its goals. Common applications of Agentic AI include robotics, where autonomous robots perform complex tasks such as manufacturing, exploration, and logistics. The defining characteristic of Agentic AI is its clear objective that guides its actions, distinguishing it from other AI types that may lack such a focused purpose. Overall, Agentic AI systems are designed to act independently and intelligently in dynamic environments.'